In [28]:
import os
import time
import requests
import pandas as pd
import json

api_key = os.environ.get('riot_api_key')
base_url = "https://na1.api.riotgames.com"
solo_duo = "RANKED_SOLO_5x5"

print(api_key)
print(base_url)

RGAPI-bd82de61-c8fa-49e2-843e-fdb603f231b9
https://na1.api.riotgames.com


In [29]:
def getLadder(type) -> pd.DataFrame:
	url = f"{base_url}/lol/league/v4/{type}/by-queue/{solo_duo}?api_key={api_key}"

	response = requests.get(url)

	print(response)

	if response.status_code == 200:
			data = response.json()
			players = data['entries']

			return pd.DataFrame(players)


In [30]:
def getMatchIDs(puuid):
	url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&start=0&count=40&api_key={api_key}"

	response = requests.get(url)

	if response.status_code == 200:
			data = response.json()

			return data
	else:
		print(response)

In [31]:
blueTeamId = 100
redTeamId = 200
def getWinner(teams):
	for t in teams:
		if t['teamId'] == blueTeamId:
			return 0 if t['win'] else 1

def getChampionID(participants, side, position):
	for p in participants:
		if p['teamId'] == side and p['teamPosition'] == position:
			return p['championId']


def mapData(matchId):
	url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}"

	response = requests.get(url)

	if response.status_code == 200:
		match_data = response.json()

		scrubbed_match_data = {
			'matchId': match_data['metadata']['matchId'],
			'gameDurationSeconds': match_data['info']['gameDuration'],
			'championBlueTop': getChampionID(match_data['info']['participants'], blueTeamId, 'TOP'),
			'championBlueJG': getChampionID(match_data['info']['participants'], blueTeamId, 'JUNGLE'),
			'championBlueMid': getChampionID(match_data['info']['participants'], blueTeamId, 'MIDDLE'),
			'championBlueBot': getChampionID(match_data['info']['participants'], blueTeamId, 'BOTTOM'),
			'championBlueSup': getChampionID(match_data['info']['participants'], blueTeamId, 'UTILITY'),
			'championRedTop': getChampionID(match_data['info']['participants'], redTeamId, 'TOP'),
			'championRedJG': getChampionID(match_data['info']['participants'], redTeamId, 'JUNGLE'),
			'championRedMid': getChampionID(match_data['info']['participants'], redTeamId, 'MIDDLE'),
			'championRedBot': getChampionID(match_data['info']['participants'], redTeamId, 'BOTTOM'),
			'championRedSup': getChampionID(match_data['info']['participants'], redTeamId, 'UTILITY'),
			'winner': getWinner(match_data['info']['teams']) # 0 = Blue side, 1 = Red Side
		}

		return scrubbed_match_data

In [32]:
def getAllLadders():
	ladder = pd.concat([getLadder("challengerleagues"), getLadder("grandmasterleagues"), getLadder("masterleagues")])

	return ladder

In [33]:
def getMatchIDs(ladder):
	match_ids = []
	puuidProcessed = []

	try:
		for index, row in ladder.iterrows():
			matches = getMatchIDs(row.puuid)
			match_ids.extend(matches)
			puuidProcessed.append(row.puuid)

			if (index + 1) % 20 == 0:
				print(len(match_ids))
				time.sleep(30) # Sleep 30 seconds so that it won't hit the rate limit

		print(len(match_ids)) # should be 6000


	except Exception as e: print(e)
	finally:
		df = pd.DataFrame({ 'match_ids': match_ids })
		df2 = pd.DataFrame({ 'puuid': puuidProcessed })

		print(df.shape)
		print(df2.shape)

		df.to_csv("./MatchIDs2.csv")
		df2.to_csv("./ProcessedPuuids2.csv")

In [34]:
# Remove Duplicates
match_ids = pd.read_csv("MatchIDs.csv")
match_ids = match_ids.drop("Unnamed: 0", axis=1)

print(match_ids.shape)

match_ids = match_ids.drop_duplicates()

print(match_ids.shape)


(236908, 1)
(81542, 1)


In [35]:
def getMatches(match_ids):
	allMatches = []
	matchIdProcessed = []
	try:

		for index, row in match_ids.iterrows():
			matchId = row.match_id

			url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}"
			response = requests.get(url)
			
			if response.status_code == 200:
				match_data = response.json()
				allMatches.append(match_data)

			matchIdProcessed.append(matchId)
			if (index + 1) % 100 == 0:
				print(len(allMatches))				
				time.sleep(121) # Sleep 121 seconds so that it won't hit the rate limit

			if (index + 1) % 10000 == 0:
				print(index + 1, 'Dumping to JSON')
				with open('match_data.json', 'w') as file:
					json.dump(allMatches, file)


		with open('match_data.json', 'w') as file:
			json.dump(allMatches, file)
	
	except Exception as e: print(e)
	finally:
		df = pd.DataFrame({ 'match_id': matchIdProcessed })
		df.to_csv("./MatchIDsProcessed2.csv")

		with open('match_data2.json', 'w') as file:
			json.dump(allMatches, file)

In [36]:
# Remove already processed
processedMatchIDs = pd.read_csv('MatchIDsProcess.csv')

print(processedMatchIDs.shape)
print(match_ids.shape)

match_ids = match_ids[~match_ids['match_id'].isin(processedMatchIDs['match_id'])]

match_ids.shape

(1757, 2)
(81542, 1)


(79785, 1)

In [37]:
getMatches(match_ids)

100
142
209
307
356
417
476
528
558
658
706
792
871
946
1000
1100
1200
1257
1304
1398
1436
1493
1574
1660
1711
1811
1853
1876
1976
2010
2081
2148
2248
2348
2392
2447
2505
2600
2670
2770
2830
2882
2938
3020
3109
3139
3204
3240
3279
HTTPSConnectionPool(host='americas.api.riotgames.com', port=443): Read timed out. (read timeout=None)


In [38]:
# data = None
# with open('match_data.json', 'r') as file:
#     data = json.load(file)

# # Print the data
# print(len(data))
# print(data[1500])